In [ ]:
#%pip install gensim

In [ ]:
#%pip install spacy

In [ ]:
#%pip install pyLDAvis 

In [46]:
#Imports
import pandas as pd
import pickle
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('words')
import pandas as pd
from nltk.tokenize import PunktSentenceTokenizer, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt

import gensim
from gensim.models.ldamodel import LdaModel
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim import matutils

import spacy

import pyLDAvis
import pyLDAvis.gensim_models

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\louis\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
#Import soptify + genius dataset with song lyrics
df_lyrics = pd.read_pickle("df_rap_7k.pickle")

In [3]:
df_lyrics.head()

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,track_release_year,track_release_decade,artist_name,artist_genres,artist_followers,artist_popularity,lyrics,lyrics_clean,lyrics_clean_split,lyrics_wordlen
0,3SUiKUnZlKYliid0ZlCmUY,0uYanB3WWNU2lT9AYxCstH,3M1quhETLChtt9NHKVaajX,189405,True,https://api.spotify.com/v1/tracks/3SUiKUnZlKYl...,Detroit Flow,44,track,0.793,...,2020,20,Quin Nfn,"[austin hip hop, southern hip hop, vapor trap]",257151,61,"Detroit Flow Lyrics\n(Ayy, DJ Showoutt, fuck g...",detroit flow fuck hold really like sound beats...,"[detroit, flow, fuck, hold, really, like, soun...",244
1,2jdWfnwHSZLuBFzsA6axWt,0iNKw3iqIEy2dh7RUdKzjY,0nQlnRkAhiWnSjTmB4JkfK,273099,True,https://api.spotify.com/v1/tracks/2jdWfnwHSZLu...,Handz in da Air,8,track,0.826,...,2018,10,Sons of Light,[],141,7,Pet Sound LyricsTurbid:\nYes it's Turbid and I...,pet sound yes turbid like reclaim freely like ...,"[pet, sound, yes, turbid, like, reclaim, freel...",209
2,2vcLUuRT3sqXnPJaIX5w5c,0VptfDjz26NkJS4b7bjmNK,0FtsMKmZEq8fBWqdSOWtqp,236619,True,https://api.spotify.com/v1/tracks/2vcLUuRT3sqX...,Substance (We Woke Up),20,track,0.731,...,2021,20,03 Greedo,"[cali rap, melodic rap, trap, vapor trap]",317202,71,"Substance (We Woke Up) Lyrics\nYeah, 03\nYeah,...",substance woke yeah yeah yeah yeah beat boy ta...,"[substance, woke, yeah, yeah, yeah, yeah, beat...",269
3,1r0faljjM2b876iNoaDUh5,0XAIjjN5qxViVS0Y5fYkar,0c173mlxpT3dSFRgMO8XPh,287320,True,https://api.spotify.com/v1/tracks/1r0faljjM2b8...,Sacrifices,56,track,0.779,...,2017,10,Big Sean,"[detroit hip hop, hip hop, pop, pop rap, rap, ...",9276030,84,"Sacrifices Lyrics\nBrrrt!\nI look up, so many ...",look many shit many young metro trust gon shoo...,"[look, many, shit, many, young, metro, trust, ...",421
4,3yhatBRYsMwuozCdQsGiTS,2t70lfxoIbKZHKFKN9ZnqE,0auu2itHTxEdAMRHvx7CyG,283200,False,https://api.spotify.com/v1/tracks/3yhatBRYsMwu...,Hard Core Rap Act,28,track,0.813,...,1995,90,Das EFX,"[east coast hip hop, gangster rap, hardcore hi...",301411,60,Hardcore Rap Act LyricsIntro/Hook (x2)\n(*The ...,hardcore rap act hook hardcore rap act back ha...,"[hardcore, rap, act, hook, hardcore, rap, act,...",267


In [4]:
#Create lemmatizer, stemmer and define stopwords

word_tokenizer=RegexpTokenizer(r'\w+')

wordnet_lemmatizer = WordNetLemmatizer()

stemmer = PorterStemmer()

stopwords_nltk = set(stopwords.words('english'))



In [5]:
#Save lyrics
temp = df_lyrics["lyrics_clean"].copy()

In [6]:
### Function to lemmatize tokenized documents (song lyrics here)
def lemmatizer(df):
    
    lemmed = []
    
    for song in df.lyrics_clean_split:
        lemmed.append([wordnet_lemmatizer.lemmatize(word) for word in song])
    return lemmed

In [173]:
### Fucntion to get dictionary and bag of words
#Returns dictionary and bow corpus
def get_dict_and_bow(data, extreme_list):
    
    #Create dictionary of words: mapping between word and id
    id2word = corpora.Dictionary(data)
    id2word.filter_extremes(no_below = extreme_list[0], no_above = extreme_list[1]) #Filter tokens that appear in less than and more than
    texts = data
    
    #Transform dictionary into bag of words
    corpus = [id2word.doc2bow(text) for text in texts]
    
    return id2word, corpus

In [19]:
### Function to get coherence score for particular model, dict and bow
#Returns the coherence model
def get_coherence(model, text, id_dict, method='c_v'):
    coherence_model = CoherenceModel(model=model, texts=text, dictionary=id_dict, coherence=method)
    coherence_lda = coherence_model.get_coherence()
    print(coherence_lda)
    return coherence_model

In [124]:
### Function to join document-topic scores back to document dataset
#Args: dataframe to join back into, corpus/bow, model to extract topics from 
def get_topics_in_df(df, corpus, model):

    #set minimum probability to 0 to ensure all topic scores are extracted
    get_document_topics = [model.get_document_topics(item,minimum_probability =0) for item in corpus]

    temp_df = []
    for i in range(len(get_document_topics)):
        temp_list = []
        for j in range(len(get_document_topics[i])):
            temp_tuple = get_document_topics[i][j]
            temp_list.append(temp_tuple[1])
        temp_df.append(temp_list)
    temp_df = pd.DataFrame(temp_df)
    df = df.join(temp_df)
    
    return df

In [9]:
lyrics_lemmatized_all = lemmatizer(df_lyrics)

In [131]:
#Create new column in df_lyrics with lemmed words
df_lyrics['lyrics_clean_split_lemmed'] = lyrics_lemmatized_all

In [174]:
id2word_all, corpus_all = get_dict_and_bow(lyrics_lemmatized_all, [200,0.95])

In [183]:
#Fit LDA model
lda_model_all = LdaModel(corpus = corpus_all,
                    id2word = id2word_all,
                    num_topics = 8, 
                    random_state = 123,
                    passes=3,
                    alpha='auto',
                    eta='auto',
                    per_word_topics=True)

In [184]:
#Show the LDA topics: topic 0
lda_model_all.show_topic(topicid=0, topn=20)


[('nigga', 0.047389094),
 ('got', 0.032168664),
 ('like', 0.029986052),
 ('shit', 0.02935791),
 ('get', 0.02806116),
 ('bitch', 0.025469694),
 ('fuck', 0.02541412),
 ('back', 0.016106253),
 ('know', 0.0150675),
 ('hit', 0.010607742),
 ('gon', 0.010162728),
 ('money', 0.008876171),
 ('cause', 0.00866876),
 ('make', 0.00836069),
 ('put', 0.008160906),
 ('as', 0.007954941),
 ('real', 0.0071295374),
 ('come', 0.0068483707),
 ('bout', 0.0068449955),
 ('see', 0.0062882677)]

In [185]:
#Get coherence score
coherence_model_all = get_coherence(lda_model_all, text = lyrics_lemmatized_all, id_dict=id2word_all)

0.4253029118240076


In [186]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis_all = pyLDAvis.gensim_models.prepare(lda_model_all, corpus_all, id2word_all)
vis_all

C:\Users\louis\anaconda3\envs\INSY669\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.069596 -0.071861       1        1  18.725581
4     -0.060991  0.023943       2        1  17.938285
3     -0.068231  0.057165       3        1  15.447304
2     -0.076241 -0.150840       4        1  12.667152
7     -0.069797  0.015812       5        1  11.189513
6      0.319100 -0.048472       6        1  10.356350
5      0.084300  0.109434       7        1   9.497115
1     -0.058544  0.064819       8        1   4.178701, topic_info=      Term          Freq         Total Category  logprob  loglift
400   feat  34534.000000  34534.000000  Default  30.0000  30.0000
173   yeah  27750.000000  27750.000000  Default  29.0000  29.0000
68     one  10118.000000  10118.000000  Default  28.0000  28.0000
64   nigga  15025.000000  15025.000000  Default  27.0000  27.0000
2    bitch  12671.000000  12671.000000  Default  26.0000  26.0000
..     ...           ...           ...      ...      ...      ...
50     let    493.034139   8573.725719   Topic8  -4.6787   0.3193
35     got    637.299502  23583.667972   Topic8  -4.4221  -0.4359
51    like    696.462361  34327.971574   Topic8  -4.3333  -0.7225
47    know    507.039391  18948.823405   Topic8  -4.6507  -0.4457
55    make    355.102639   8279.961725   Topic8  -5.0069   0.0260

[477 rows x 6 columns], token_table=      Topic      Freq   Term
term                        
874       1  0.035328    ace
874       2  0.054955    ace
874       3  0.003925    ace
874       4  0.023552    ace
874       5  0.003925    ace
...     ...       ...    ...
114       4  0.067422  young
114       5  0.067422  young
114       6  0.575720  young
114       7  0.069630  young
114       8  0.017662  young

[2588 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 4, 3, 8, 7, 6, 2])

In [154]:
#Join topics back to dataframe
df_lyrics_topics = df_lyrics.copy()
df_lyrics_topics = get_topics_in_df(df = df_lyrics, corpus = corpus_all, model = lda_model_all)

In [155]:
df_lyrics_topics

,track_id,album_id,artist_id,duration_ms,flg_explicit,url,track_name,track_popularity,track_type,track_danceability,...,lyrics_wordlen,lyrics_clean_split_lemmed,0,1,2,3,4,5,6,7
0,3SUiKUnZlKYliid0ZlCmUY,0uYanB3WWNU2lT9AYxCstH,3M1quhETLChtt9NHKVaajX,189405,True,https://api.spotify.com/v1/tracks/3SUiKUnZlKYl...,Detroit Flow,44,track,0.793,...,244,"[detroit, flow, fuck, hold, really, like, soun...",0.169173,0.148124,0.004778,0.001348,0.415081,0.001422,0.100449,0.159626
1,2jdWfnwHSZLuBFzsA6axWt,0iNKw3iqIEy2dh7RUdKzjY,0nQlnRkAhiWnSjTmB4JkfK,273099,True,https://api.spotify.com/v1/tracks/2jdWfnwHSZLu...,Handz in da Air,8,track,0.826,...,209,"[pet, sound, yes, turbid, like, reclaim, freel...",0.266172,0.448152,0.184567,0.041370,0.001710,0.001661,0.001653,0.054715
2,2vcLUuRT3sqXnPJaIX5w5c,0VptfDjz26NkJS4b7bjmNK,0FtsMKmZEq8fBWqdSOWtqp,236619,True,https://api.spotify.com/v1/tracks/2vcLUuRT3sqX...,Substance (We Woke Up),20,track,0.731,...,269,"[substance, woke, yeah, yeah, yeah, yeah, beat...",0.574550,0.044220,0.015518,0.030843,0.009153,0.001357,0.323100,0.001259
3,1r0faljjM2b876iNoaDUh5,0XAIjjN5qxViVS0Y5fYkar,0c173mlxpT3dSFRgMO8XPh,287320,True,https://api.spotify.com/v1/tracks/1r0faljjM2b8...,Sacrifices,56,track,0.779,...,421,"[look, many, shit, many, young, metro, trust, ...",0.322452,0.007881,0.022331,0.012470,0.198162,0.003536,0.017884,0.415286
4,3yhatBRYsMwuozCdQsGiTS,2t70lfxoIbKZHKFKN9ZnqE,0auu2itHTxEdAMRHvx7CyG,283200,False,https://api.spotify.com/v1/tracks/3yhatBRYsMwu...,Hard Core Rap Act,28,track,0.813,...,267,"[hardcore, rap, act, hook, hardcore, rap, act,...",0.001252,0.812703,0.001194,0.001211,0.035724,0.001211,0.145434,0.001271
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6778,3YzxXVNHGZF09eGqhsYuHI,6CM5qhYBvpgYNek5kYwuOJ,20qISvAhX20dpIbOOzGK3q,187175,True,https://api.spotify.com/v1/tracks/3YzxXVNHGZF0...,The Pressure,56,track,0.573,...,192,"[pressure, pressure, pressure, feel, pressure,...",0.083606,0.623860,0.143020,0.001649,0.001725,0.001663,0.001694,0.142783
6779,7KIbDUwumrpG5f30kEYW1v,6Gdt5ogiuJ9knp8Q5148ea,1yxSLGMDHlW21z4YXirZDS,213066,False,https://api.spotify.com/v1/tracks/7KIbDUwumrpG...,Pump It,0,track,0.648,...,179,"[pump, pump, pump, pump, pump, pump, turn, rad...",0.324463,0.390463,0.001719,0.125263,0.001898,0.001730,0.152629,0.001834
6780,4a4dFaA4NfXU5A3snq2ZMa,4MloRVOQvX5YAfu2ik7OA9,3BeQqzKdlARoOd6y30kCO2,263773,True,https://api.spotify.com/v1/tracks/4a4dFaA4NfXU...,Made Man,12,track,0.647,...,184,"[made, man, singing, background, blessing, liv...",0.001884,0.711127,0.001759,0.001759,0.001824,0.001872,0.001827,0.277948
6781,2HIjQZsmscClqRGbfzwAgK,24jAbLMFzCTlIv3kQ94HwK,50co4Is1HCEo8bhOyUWKpn,210733,True,https://api.spotify.com/v1/tracks/2HIjQZsmscCl...,Diamonds (feat. Gunna),55,track,0.919,...,256,"[highest, lowest, young, stoner, life, stoner,...",0.001258,0.001207,0.001218,0.001231,0.007159,0.985498,0.001213,0.001215


In [ ]:
#Export songs to pickle 
#df_lyrics_topics.to_pickle("df_lyrics_topics.pickle")